# EC3310 PS6


**Due 10-MAR-2021**

In [18]:
import pandas as pd  # Import pandas and use shorthand notation pd to save on typing
# can also import Series and DataFrame into the local namespace to save on typing pd. every time:
from pandas import Series, DataFrame 
import numpy as np                            # Some housekeeping
import statsmodels.api as sm # import statsmodels 

import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
import seaborn as sns

import requests
import json
from datetime import datetime


## S&P 500 and Days of the Week

**0.** Get your very own AlphaVantage API key [here](https://www.alphavantage.co/support/#api-key).

**1.** Use the AlphaVantage to create a dataframe with the historical values for the S&P500 (use the stock ticker symbol `IVV` which is an ETF that tracks the whole index). Set "outputsize" to "full" to get the entire historical record.

In [23]:
x="IVV"
api='https://www.alphavantage.co/query?'
data= {"function":"time_series_daily","symbol":x,"outputsize":"full","apikey":"ET9DLN821TXY76P8",}
response=requests.get(api, params=data)


requests.models.Response

**2.** Create a new column `movement` with the difference between the opening and closing prices. 

In [24]:
ticker_dict = json.loads(response.content)
frame = pd.DataFrame(ticker_dict['Time Series (Daily)']).T 
frame.name = "IVV stock prices"
frame.columns = list(map(lambda x: x[3:], frame.columns))
frame.open = frame.open.astype(float)
frame.close = frame.close.astype(float)
#frame.volume = frame.volume.astype(float)
frame['movement']=frame['close']-frame['open']
frame

,open,high,low,close,volume,movement
2021-03-23,394.69,396.2400,391.4100,392.22,5838467,-2.47
2021-03-22,392.82,396.8600,392.7600,395.30,3791578,2.48
2021-03-19,392.68,394.3300,389.9200,392.20,6090840,-0.48
2021-03-18,395.99,398.2400,392.2500,392.93,4827129,-3.06
2021-03-17,396.04,399.6500,394.8100,398.75,4506225,2.71
...,...,...,...,...,...,...
2000-05-25,140.00,140.9000,137.9000,138.50,69600,-1.50
2000-05-24,137.80,140.1000,136.7000,139.80,400300,2.00
2000-05-23,140.20,140.2000,137.7000,137.70,373900,-2.50
2000-05-22,140.60,140.6000,136.8000,139.80,1850600,-0.80


**3.** Then create a new column `weekday`, that has the day of the week (Monday, Tuesday, etc) as **strings**. To do this, you must first create a python datetime object from the string date (the index of the dataframe). Then you can use the `.weekday()` method on this datetime object to get the day of the week. The output of the `.weekday()` method is an integer 0-6 with 0 corresponding to Monday and 6 Sunday. 

In [13]:
def week_day(str_date):
    dotw=datetime.strptime(str_date, '%Y-%m-%d').weekday() #Obtains the day of the week for the input date
    days=["Monday","Tuesday","Wednesday","Thursday","Friday"] #List of days to use as outputs
    for n in range(6):  #Iterates 7 times
        if dotw==n: #Tests if the output mataches the nth day
            return days[n] 
frame['weekday'] = frame.index.map(week_day) #Appends the weekday column on to the dataframe
frame


,open,high,low,close,volume,movement,weekday
2021-03-23,394.69,396.2400,391.4100,392.22,5838163,-2.47,Tuesday
2021-03-22,392.82,396.8600,392.7600,395.30,3791578,2.48,Monday
2021-03-19,392.68,394.3300,389.9200,392.20,6090840,-0.48,Friday
2021-03-18,395.99,398.2400,392.2500,392.93,4827129,-3.06,Thursday
2021-03-17,396.04,399.6500,394.8100,398.75,4506225,2.71,Wednesday
...,...,...,...,...,...,...,...
2000-05-25,140.00,140.9000,137.9000,138.50,69600,-1.50,Thursday
2000-05-24,137.80,140.1000,136.7000,139.80,400300,2.00,Wednesday
2000-05-23,140.20,140.2000,137.7000,137.70,373900,-2.50,Tuesday
2000-05-22,140.60,140.6000,136.8000,139.80,1850600,-0.80,Monday


**4.** Construct an OLS model regressing the day of the week on movement. To do this you must create weekday dummy variables. Do *not* add a constant.

In [5]:
frame_2 = pd.concat((frame, pd.get_dummies(frame['weekday'], drop_first=True)), axis=1) #Creates a dataframe with n-1 dummies
x = frame_2[["Monday","Tuesday","Wednesday","Thursday"]] #Sets the inputs of x and y
y = frame_2["movement"]
ols = sm.OLS(y, x) #Calls the OLS function
ols.fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:               movement   R-squared (uncentered):                   0.000
Model:                            OLS   Adj. R-squared (uncentered):             -0.001
Method:                 Least Squares   F-statistic:                             0.1964
Date:                Wed, 17 Mar 2021   Prob (F-statistic):                       0.940
Time:                        05:32:28   Log-Likelihood:                         -9708.5
No. Observations:                5238   AIC:                                  1.942e+04
Df Residuals:                    5234   BIC:                                  1.945e+04
Df Model:                           4                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Monday         0.0021      0.049      0.043      0.965      -0.094       0.099
Tuesday       -0.0119      0.047     -0.252      0.801      -0.104       0.081
Wednesday      0.0241      0.047      0.511      0.609      -0.068       0.117
Thursday       0.0322      0.048      0.677      0.498      -0.061       0.125
==============================================================================
Omnibus:                     1135.230   Durbin-Watson:                   2.165
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            19724.076
Skew:                          -0.565   Prob(JB):                         0.00
Kurtosis:                      12.439   Cond. No.                         1.04
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

**5.** Find the average movement across all observations. Call this `avg`.

Next, construct a new data frame with the average movement of each day of the week. That is, a data frame with rows corresponding to the days of the week and with 1 column `movement` that returns the average movement over all observation of that day of the week minus `avg` (that is the weekday idiosyncratic mean). For example the 'Tuesday' should have the average over all movement on tuesdays minus the average movement over all days.

Finally, add another column `coeff` that has the coefficients on the dummy variables from the last part of the question. Note there will be one weekday missing since we have N-1 dummies for an N-category variable. Find the difference between these two columns.

In [61]:
avg=frame['movement'].mean() #Calculates the average for the entire column
#print(avg)
mon_list=[] #Empty lists 
tue_list=[]
wed_list=[]
thur_list=[]
fri_list=[]
for n in range(len(frame)): #Iterates over the enitre DataFrame
    if frame['weekday'][n] == "Monday": #Tests if the weekday column matches the specified day of the week
        mon_list.append(frame['movement'][n]) #Appends the movement for that date onto the relevant list
    if frame['weekday'][n] == "Tuesday":
        tue_list.append(frame['movement'][n])
    if frame['weekday'][n] == "Wednesday":
        wed_list.append(frame['movement'][n])
    if frame['weekday'][n] == "Thursday":
        thur_list.append(frame['movement'][n])
    if frame['weekday'][n] == "Friday":
        fri_list.append(frame['movement'][n])
        
mon_avg=sum(mon_list)/len(mon_list)-abs(avg) #Calulates the idiosyncratic mean for each day of the week
tue_avg=sum(tue_list)/len(tue_list)-abs(avg)
wed_avg=sum(wed_list)/len(wed_list)-abs(avg)
thur_avg=sum(thur_list)/len(thur_list)-abs(avg)
fri_avg=sum(fri_list)/len(fri_list)-abs(avg)

data={"movement":{'Monday':mon_avg,'Tuesday':tue_avg,'Wednesday':mon_avg,'Thursday':thur_avg,'Friday':fri_avg}}
df=DataFrame(data ,index=["Monday","Tuesday","Wednesday","Thursday", "Friday"], columns=['movement']) #Generates dataframe

coeff=[-6.951e-05,-0.0108,0.0243,0.0310,0.0] #Copied from  the answer of Q4 (Fridays value has been taken as 0)
df['coeff'] = coeff #Appends the dataframe with the new list
df['difference']=df['movement']-df['coeff'] #Creates a new column with the difference as its inputs
df

,movement,coeff,difference
Monday,0.001247,-0.00007,0.001316
Tuesday,-0.012780,-0.01080,-0.001980
Wednesday,0.001247,0.02430,-0.023053
Thursday,0.031324,0.03100,0.000324
Friday,-0.052124,0.00000,-0.052124


In [62]:
df=df.drop_duplicates(subset='movement')
df

,movement,coeff,difference
Monday,0.001247,-0.00007,0.001316
Tuesday,-0.012780,-0.01080,-0.001980
Thursday,0.031324,0.03100,0.000324
Friday,-0.052124,0.00000,-0.052124


In [28]:
sym_list = ['IVV', 'GME', 'TSLA']
r=[]
for i in range(len(sym_list)):
    api='https://www.alphavantage.co/query?'
    data= {"function":"time_series_daily","symbol":sym_list[i],"outputsize":"full","apikey":"ET9DLN821TXY76P8",}
    r.append(requests.get(api, params=data))
for n in range(len(r)):
    ticker_dict = json.loads(r[n].content)
    frame = pd.DataFrame(ticker_dict['Time Series (Daily)']).T 
    frame.columns = list(map(lambda x: x[3:], frame.columns))
    frame.open = frame.open.astype(float)
    frame.close = frame.close.astype(float)
    frame['movement']=frame['close']-frame['open']
    frame['Symbol']=sym_list[n]
    del frame['high']
    del frame['low']
    del frame['volume']
frame

KeyError: 'Time Series (Daily)'